In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
import pathlib

In [2]:
def from_keras_model(kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return tflite_model
    
def write_model(tflite_model,path):
    tflite_model_files = pathlib.Path(path)
    tflite_model_files.write_bytes(tflite_model)

In [3]:
# load the model
model = tf.keras.models.load_model(
    "resources/cassava_model.h5"
)

# print the model summary to see it shape
model.summary()

2023-01-15 11:43:41.077384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 16, 16, 1280)     2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 320)               409920    
                                                                 
 dropout (Dropout)           (None, 320)               0         
                                                                 
 dense_1 (Dense)             (None, 320)               102720    
                                                                 
 dropout_1 (Dropout)         (None, 320)               0

In [4]:
num_disease_map={
"0":"Cassava Bacterial Blight (CBB)",
"1":"Cassava Brown Streak Disease (CBSD)",
"2":"Cassava Green Mottle (CGM)",
"3":"Cassava Mosaic Disease (CMD)",
"4":"Healthy",
}

In [8]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
# load the images
img = load_img('resources/test_image_2.jpg', target_size=(512, 512))
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
# preprocess image
img = img/255.0

In [9]:
prediction = model.predict(img)

1/1 [==============================] - 1s 1s/step


In [10]:
prediction[0]

array([0.25404036, 0.20054367, 0.12246692, 0.24141572, 0.18153328],
      dtype=float32)

In [11]:
index = np.argmax(prediction[0])
disease_predicted = num_disease_map[str(index)]

In [12]:
disease_predicted

'Cassava Bacterial Blight (CBB)'

In [13]:
cassava_tflite_model = from_keras_model(model)

INFO:tensorflow:Assets written to: /var/folders/1d/bsb5054s6jqfmkmqs04j7lmh0000gn/T/tmpt7ctavu3/assets


INFO:tensorflow:Assets written to: /var/folders/1d/bsb5054s6jqfmkmqs04j7lmh0000gn/T/tmpt7ctavu3/assets
2023-01-15 12:54:44.959646: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-15 12:54:44.960224: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-15 12:54:44.966164: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/1d/bsb5054s6jqfmkmqs04j7lmh0000gn/T/tmpt7ctavu3
2023-01-15 12:54:45.012008: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-15 12:54:45.012035: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/1d/bsb5054s6jqfmkmqs04j7lmh0000gn/T/tmpt7ctavu3
2023-01-15 12:54:45.150902: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-01-15 12:54:45.184565: I tensorflow/cc/saved_model/load

In [14]:
write_model(cassava_tflite_model,'resources/cassava_model.tflite')